In [4]:
import numpy as np
import gym
import random
import math

In [12]:
env1 =["SHFF", "FFFH", "FHFH", "HFFG"]
env2= ["SFFFFF", "FFFHFF", "FHFHHH", "HFFFFG"]
env3 = ['SFFHFFHH', 'HFFFFFHF', 'HFFHHFHH', 'HFHHHFFF', 'HFHHFHFF', 'FFFFFFFH', 'FHHFHFHH', 'FHHFHFFG'] 

selectedEnv = env2
env = gym.make('FrozenLake-v1', desc=selectedEnv, render_mode="human", is_slippery = False)
env.reset()
env.render()

In [7]:
# change-able parameters:
discount_factor = 0.99
delta_threshold = 0.00001
epsilon = 1

In [8]:
def value_iteration(env, gamma=0.9, epsilon=1e-6):
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    # Initialize the value function
    V = np.zeros(num_states)

    #Write your code to implement value iteration main loop



         
    # Write your code here to extract the optimal policy from value function. 
    # For each state, the policy will tell you the action to take
    policy = np.zeros(num_states, dtype=int)
    
       
    return policy, V


In [13]:
# Run value iteration
policy, V = value_iteration(env)

# Print results
print("Optimal Value Function:")
print(V.reshape(len(selectedEnv), len(selectedEnv[0])))

print("\nOptimal Policy (0=Left, 1=Down, 2=Right, 3=Up):")
print(policy.reshape(len(selectedEnv), len(selectedEnv[0])))

Optimal Value Function:
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]

Optimal Policy (0=Left, 1=Down, 2=Right, 3=Up):
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [14]:
# resetting the environment and executing the policy
state = env.reset()
state = state[0]
step = 0
done = False
print(state)

max_steps = 100
for step in range(max_steps):

    # Getting max value against that state, so that we choose that action
    action =policy[state]
    new_state, reward, done, truncated, info = env.step(action) #information after taking the action

    env.render()
    if done:
        print("number of steps taken:", step)
        break

    state = new_state

env.close()

0


KeyboardInterrupt: 